# Validation of OpenBioLink Network
This notebook maps the drug-disease pairs (DrugBank-MeSH) from ClinicalTrials.gov (16-04-2020)
to DO for validating the OpenBioLink network.

In [1]:
import getpass
import sys
import time

import urllib.request
import json 

import pandas as pd

In [2]:
getpass.getuser()

'danieldomingo'

In [3]:
sys.version

'3.7.4 (v3.7.4:e09359112e, Jul  8 2019, 14:54:52) \n[Clang 6.0 (clang-600.0.57)]'

In [4]:
time.asctime()

'Mon Apr 20 16:46:47 2020'

Load clinical trials information from ClinicalTrials.gov

In [5]:
clinical_trials = pd.read_csv(
    "../data/PubChem-MeSH-slim-counts.tsv",
    sep="\t",
)

Load Disease Ontology Cross-references

In [6]:
with urllib.request.urlopen(
    "https://github.com/DiseaseOntology/HumanDiseaseOntology/raw/master/src/ontology/doid.json"
) as url:
    do_data = json.loads(url.read().decode())

In [7]:
def get_mesh_to_do_mappings(do_data):
    """Parse Disease Ontology file to get mappings between MeSH terms and Disease Ontology."""
    meshid_to_doid = {}
    
    # Multiple graphs within the file
    for graph in do_data['graphs']:

        # Iterate for each node in the graph
        for node in graph['nodes']:
            # Check if there is metadata in the node
            if "meta" in node and "xrefs" in node["meta"]:
                # Remove prefix from URL to get the ID
                doid = node["id"].replace("http://purl.obolibrary.org/obo/", "").split("_")[1]

                for xref in node["meta"]["xrefs"]:
                    # Check the resource (MeSH, UMLS, etc...)
                    source = xref['val'].split(":")[0]
                    
                    # Skip non MESH crossrefs
                    if source != "MESH":
                        continue

                    meshid_to_doid[xref['val'].split(":")[1]] = f"DOID:{doid}"

    return meshid_to_doid

In [8]:
meshid_to_doid = get_mesh_to_do_mappings(do_data)

print(f"Total of mappings between MeSH and DO: {len(meshid_to_doid)}")

Total of mappings between MeSH and DO: 3154


Diseases mapped between OpenBioLink (DO) and ClinicalTrials.gov (MeSH)

In [9]:
mapped_meshs = set(meshid_to_doid.keys()).intersection(set(clinical_trials.condition.unique()))

In [10]:
len(mapped_meshs)

1603

OpenBioLink

In [11]:
openbiolink_df = pd.read_csv(
    "../../networks/data/openbiolink_network.csv",
)

In [12]:
openbiolink_df.head()

,SOURCE,TARGET,BEL_RELATION
0,DOID:0050156,HP:0002206,increases
1,DOID:0050156,HP:0100759,increases
2,DOID:0050158,HP:0002878,increases
3,DOID:0050158,HP:0003593,increases
4,DOID:0050158,HP:0006515,increases


Get DOID target nodes

In [13]:
openbiolink_disease_nodes = {
    row[1]
    for row in openbiolink_df.itertuples(index=False)
    if "DOID" in row[1] and "NCBIGENE" in row[0]
}
print(f"Diseases in OpenBioLink from Disease Ontology that are target by a gene: {len(openbiolink_disease_nodes)}")

Diseases in OpenBioLink from Disease Ontology that are target by a gene: 1512


Get PubChem source nodes

In [14]:
openbiolink_drug_source_nodes = {
    int(row[0].replace("PUBCHEM.COMPOUND:", "")) # Cast to int
    for row in openbiolink_df.itertuples(index=False)
    if "PUBCHEM.COMPOUND:" in row[0]
}
print(f"Drugs in OpenBioLink from PubChem that can be used as source nodes: {len(openbiolink_drug_source_nodes)}")

Drugs in OpenBioLink from PubChem that can be used as source nodes: 7407


In [15]:
pairs_for_validation = pd.DataFrame([
    # pubchem and mapped DO terms
    {'cid_id': row['cid_id'], 'do_id': meshid_to_doid[row['condition']], 'n_trials': row['n_trials']}
    for _, row in clinical_trials.iterrows()
    if row['condition'] in meshid_to_doid and meshid_to_doid[row['condition']] in openbiolink_disease_nodes and row['cid_id'] in openbiolink_drug_source_nodes
])

In [16]:
pairs_for_validation.head()

,cid_id,do_id,n_trials
0,10114,DOID:1588,1
1,10133,DOID:10588,1
2,10133,DOID:9352,1
3,10133,DOID:10582,1
4,10133,DOID:905,1


In [17]:
pairs_for_validation.shape

(5151, 3)

Export drug-disease pairs for validation of the OpenBioLink network

In [18]:
pairs_for_validation.to_csv("validation_openbiolink.tsv", sep="\t", index=False)

Export source nodes for validation of the OpenBioLink network (drugs)


In [19]:
pd.Series(
    [f"PUBCHEM.COMPOUND:{cid}" for cid in  pairs_for_validation.cid_id.unique()] # add the prefix
).to_csv("../data/source_nodes_openbiolink.tsv", sep="\t", index=False)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


Export target nodes for validation of the OpenBioLink network (diseases)


In [20]:
pd.Series(
    pairs_for_validation.do_id.unique()
).to_csv("../data/target_nodes_openbiolink.tsv", sep="\t", index=False)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


## Analysis of the disease-gene associations
In the following cells, we analyzed the number of interactions for each disease (target node) to assess whether there are genes that act as funnel (most of the paths will go through that gene to the disease)

In [21]:
diseases_to_validate = pairs_for_validation.do_id.unique()

print(f"Number of diseases to validate in OpenBioLink: {len(diseases_to_validate)}")

last_relations = pd.DataFrame([
    {'do_id': row[1], 'ncbigene': row[0]}
    for row in openbiolink_df.itertuples(index=False)
    # Check that it is a final interaction (gene to disease) and the disease is in clinical trials
    if "DOID" in row[1] and "NCBIGENE" in row[0] and row[1] in diseases_to_validate
])

Number of diseases to validate in OpenBioLink: 264


In [22]:
last_relations.shape

(528, 2)

In [23]:
last_relations.groupby('do_id').count()

,ncbigene
do_id,
DOID:0050144,3
DOID:0050156,1
DOID:0050424,2
DOID:0050434,1
DOID:0050440,2
DOID:0050444,1
DOID:0050445,1
DOID:0050449,1
DOID:0050450,1
